<a href="https://colab.research.google.com/github/busiri/busil/blob/main/4%EB%8B%A8%EA%B3%84(Feature_selection).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings, sys, os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)   # 열 생략 없이 전부 출력
pd.set_option("display.expand_frame_repr", False)  # 줄 넘김 없이 한 줄에 출력

warnings.filterwarnings("ignore")

plt.rcParams["font.family"]      = "Malgun Gothic"   # Windows 기본 한글 글꼴
plt.rcParams["axes.unicode_minus"] = False           # - 부호 깨짐 방지

In [ ]:
df1=pd.read_csv('도입_t_test3.csv')
df2=pd.read_csv('성장_t_test3.csv')
df3=pd.read_csv('성숙_t_test3.csv')
df4=pd.read_csv('쇠퇴_t_test3.csv')

### LGBM을 활용하여 Feaute importance 상위 200개 피처 선정

In [ ]:

# 0) 가정: X_train, X_test, y_train, y_test 준비 완료

# 1) 스케일링
scaler = StandardScaler()
X_scaled       = scaler.fit_transform(X_train)
X_valid_scaled  = scaler.transform(X_valid)

# 2) LightGBM 모델 학습
lgb = LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    random_state=42
)
lgb.fit(X_scaled, y_train)

# 3) feature_importances_ 를 Series로 추출·정렬
feat_imp = pd.Series(
    lgb.feature_importances_,
    index=X_train.columns   # ← 여기
).sort_values(ascending=False)

# 4) 상위 K개 선택 (예: K=50)
K = 200
topK = feat_imp.head(K).index.tolist()
X_train = X_train[topK]
X_valid  = X_valid[topK]
X_test = X_test[topK]
print(f"Top {K} features:\n", feat_imp.head(K))

### t-test를 활용하여 정상집단과 부실집단 간 평균 차이를 보이는 컬럼 선정(유의수준 : 0.05)

In [ ]:
# t-test
import pandas as pd
from scipy.stats import ttest_ind

# cat_cols 미리 지정
cat_cols = [
    '회사명', '거래소코드', '회계년도', 'IFRS_CONN', 'IFRS', '기업규모명',
    '통계청 한국표준산업분류 11차(대분류)', '상장폐지일', '대표이사변경여부',
    '수도권', 'GAAP', '회계년도연속여부'
]

phase_files = [
    ('도입_train_파생_winsor.csv', '도입'),
    ('성장_train_파생_winsor.csv', '성장'),
    ('성숙_train_파생_winsor.csv', '성숙'),
    ('쇠퇴_train_파생_winsor.csv', '쇠퇴')
]

for file, label in phase_files:
    try:
        df = pd.read_csv(file)
        # 실제 데이터에 존재하는 범주형 컬럼만 추출
        existing_cat_cols = [col for col in cat_cols if col in df.columns]
        df_cat = df[existing_cat_cols].copy() if existing_cat_cols else pd.DataFrame()

        # 수치형 컬럼만 (타겟/범주형 제외)
        num_cols = df.select_dtypes(include=['number']).columns.tolist()
        if '부실여부' in num_cols:
            num_cols.remove('부실여부')
        # 범주형 변수는 t-test에서 제외

        # T-test로 p-value가 0.05 미만인 컬럼만 남김
        cols_to_keep = []
        for col in num_cols:
            group0 = df[df['부실여부'] == 0][col].dropna()
            group1 = df[df['부실여부'] == 1][col].dropna()
            if len(group0) > 1 and len(group1) > 1:
                stat, p = ttest_ind(group0, group1, equal_var=False)
                if p < 0.05:
                    cols_to_keep.append(col)
            else:
                # 샘플 부족 시 보존
                cols_to_keep.append(col)
        # 최종 컬럼: 타겟 + 유의미한 수치형
        final_cols = ['부실여부'] + cols_to_keep
        df_filtered = df[final_cols].copy()

        # t-test 후, 범주형 변수 다시 합치기 (인덱스 맞춰서)
        df_result = pd.concat([df_filtered.reset_index(drop=True), df_cat.reset_index(drop=True)], axis=1)
        df_result = df_result.loc[:, ~df_result.columns.duplicated()]

        save_name = f'{label}_t_test3.csv'
        df_result.to_csv(save_name, index=False)
        print(f"[{label}] T-test 통과 컬럼 + 범주형 변수 포함 데이터 저장 완료: {save_name}")
    except Exception as e:
        print(f"[{label}] 처리 중 에러 발생: {e}")

### 다중공선성 제거를 위해 모든 컬럼이 VIF 10 미만이 될때까지 가장 높은 VIF를 갖는 컬럼 제거

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd
import warnings

cat_cols = [
    '회사명', '거래소코드', '회계년도', 'IFRS_CONN', 'IFRS', '기업규모명',
    '통계청 한국표준산업분류 11차(대분류)', '상장폐지일', '대표이사변경여부',
    '수도권', 'GAAP', '회계년도연속여부'
]

def calculate_vif(df, target_col):
    X = df.drop(columns=[target_col], errors='ignore')
    # 수치형 컬럼만, 결측치 컬럼은 제외
    X = X.select_dtypes(include=["number"]).dropna(axis=1, how='all')
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        vif_data["VIF"] = [
            variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])
        ]
    return vif_data

def vif_removal_for_phase(df, target_col, cat_cols, threshold=10, label="국면"):
    # 1) 실제 존재하는 cat_cols만 골라오기
    existing_cats = [c for c in cat_cols if c in df.columns]
    missing = [c for c in cat_cols if c not in df.columns]
    print(f"[{label}] 누락된 cat_cols: {missing}")

    # 2) VIF 계산 대상 숫자형 컬럼에서 cat_cols 제외
    num_cols = df.select_dtypes(include=["number"]).columns.tolist()
    cols_for_vif = [
        col for col in num_cols
        if col != target_col and col not in existing_cats
    ]
    df_vif = df[cols_for_vif + [target_col]].copy()

    # 3) 존재하는 범주형 컬럼만 따로 보관
    df_cat = df[existing_cats].copy()

    # 4) VIF 반복 제거
    step = 0
    while True:
        vif_df = calculate_vif(df_vif, target_col)
        max_vif = vif_df["VIF"].max()
        max_vif_feature = vif_df.sort_values(
            by="VIF", ascending=False
        ).iloc[0]["feature"]
        print(f"[{label}] Step {step}: 최대 VIF = {max_vif:.3f} ({max_vif_feature})")
        if max_vif <= threshold:
            print(f"[{label}] 모든 VIF가 {threshold} 이하입니다. 반복 종료.")
            break
        print(
            f"[{label}] '{max_vif_feature}' 컬럼(VIF={max_vif:.3f})을 제거합니다."
        )
        df_vif = df_vif.drop(columns=[max_vif_feature])
        step += 1

    # 5) VIF에서 살아남은 숫자형 컬럼 + 타겟 + 범주형 컬럼 합치기
    df_final = pd.concat([df_vif, df_cat], axis=1)
    # 중복 컬럼 제거
    df_final = df_final.loc[:, ~df_final.columns.duplicated()]

    # 6) 결과 저장
    save_name = f'{label}_최종(컬럼선택삭제).csv'
    df_final.to_csv(save_name, index=False)
    print(f"[{label}] 최종 파일 저장 완료: {save_name}")

    return df_final, vif_df

[도입] 누락된 cat_cols: ['상장폐지일', '대표이사변경여부']
[도입] Step 0: 최대 VIF = 3736.521 (영업비용대영업수익비율)
[도입] '영업비용대영업수익비율' 컬럼(VIF=3736.521)을 제거합니다.
[도입] Step 1: 최대 VIF = 1040.847 (자기자본배율)
[도입] '자기자본배율' 컬럼(VIF=1040.847)을 제거합니다.
[도입] Step 2: 최대 VIF = 550.005 (총자본영업이익률)
[도입] '총자본영업이익률' 컬럼(VIF=550.005)을 제거합니다.
[도입] Step 3: 최대 VIF = 208.201 (총자본순이익률)
[도입] '총자본순이익률' 컬럼(VIF=208.201)을 제거합니다.
[도입] Step 4: 최대 VIF = 130.630 (타인자본구성비율)
[도입] '타인자본구성비율' 컬럼(VIF=130.630)을 제거합니다.
[도입] Step 5: 최대 VIF = 105.804 (법인세비용차감전손익(천원))
[도입] '법인세비용차감전손익(천원)' 컬럼(VIF=105.804)을 제거합니다.
[도입] Step 6: 최대 VIF = 85.825 (자산(천원))
[도입] '자산(천원)' 컬럼(VIF=85.825)을 제거합니다.
[도입] Step 7: 최대 VIF = 62.544 (유동자산구성비율)
[도입] '유동자산구성비율' 컬럼(VIF=62.544)을 제거합니다.
[도입] Step 8: 최대 VIF = 55.028 (총자본회전률)
[도입] '총자본회전률' 컬럼(VIF=55.028)을 제거합니다.
[도입] Step 9: 최대 VIF = 47.733 (총자본사업이익률)
[도입] '총자본사업이익률' 컬럼(VIF=47.733)을 제거합니다.
[도입] Step 10: 최대 VIF = 39.372 (매출액(천원))
[도입] '매출액(천원)' 컬럼(VIF=39.372)을 제거합니다.
[도입] Step 11: 최대 VIF = 38.846 (부채비율)
[도입] '부채비율' 컬럼(VIF=38.846)을 제거합니다.

In [ ]:
# 도입기 (VIF 제거)
df_final, vif_summary = vif_removal_for_phase(
    df1,
    target_col='부실여부',
    cat_cols=cat_cols,
    threshold=10,
    label="도입"
)

In [ ]:
# 성장기 (VIF 제거)
df_final2, vif_summary2 = vif_removal_for_phase(
    df2,
    target_col='부실여부',
    cat_cols=cat_cols,
    threshold=10,
    label="성장"
)

In [ ]:
# 성숙기 (VIF 제거)
df_final3, vif_summary3 = vif_removal_for_phase(
    df3,
    target_col='부실여부',
    cat_cols=cat_cols,
    threshold=10,
    label="성숙"
)

In [ ]:
# 쇠퇴기 (VIF 제거)
df_final4, vif_summary4 = vif_removal_for_phase(
    df3,
    target_col='부실여부',
    cat_cols=cat_cols,
    threshold=10,
    label="쇠퇴"
)

In [ ]:
# 파일 보내기
df_final1.to_csv('도입기_feature_selected.csv',index=False)
df_final2.to_csv('성장기_feature_selected.csv',index=False)
df_final3.to_csv('성숙기_feature_selected.csv',index=False)
df_final4.to_csv('쇠퇴기_feature_selected.csv',index=False)